# Least Squares (Scikit-Learn)

In [1]:
import pickle
import numpy as np

In [2]:
with open('../pickles/test_cells.pkl', 'rb') as f:
    cells_te = pickle.load(f)

In [3]:
def skl_cv_quad(X, y, kf=10):
    kfold = KFold(n_splits=kf, shuffle=False)
    
    te = np.hstack([cells_te+(6078*i) for i in range(5)])
    cells_tv = np.delete(range(6078), cells_te)
    tv = np.hstack([cells_tv+(6078*i) for i in range(5)])
    
    quad = np.zeros((int(X.shape[0]), int(X.shape[1]+(X.shape[1]*(X.shape[1]-1))/2)))
    quad[:,:X.shape[1]] = np.copy(X)
    col = 99
    for i in range(X.shape[1]-1):
        for j in range(i+1, X.shape[1]):
            quad[:,col] = (X[:,i]*X[:,j])
            col += 1
    
    X_te = X[te]
    y_te = y[te]
    quad_te = quad[te]
    quad_tv = quad[tv]
    y_tv = y[tv]
    
    w_list = []
    bias_list = []
    for (cell_tr, cell_v) in (kfold.split(range(5470))):
        v = np.hstack([cell_v+(5470*i) for i in range(5)])
        tr = np.hstack([cell_tr+(5470*i) for i in range(5)])
        quad_tr, y_tr = quad_tv[tr], y_tv[tr]
        quad_v, y_v = quad_tv[v], y_tv[v]
        
        regr = linear_model.LinearRegression()
        regr.fit(quad_tr, y_tr)
        w_list.append(regr.coef_.T)
        bias_list.append(regr.intercept_)
    
    w = np.mean(w_list, axis=0)
    bias = np.mean(bias_list, axis=0)
    print(w.shape, bias.shape)
    
    yp = X_te + bias + quad_te.dot(w)
    yp[yp<0] = 0
    ya = X_te + y_te
    ferror = (np.sum(np.abs(yp - ya)**2, axis=0)/np.sum(np.abs(ya)**2, axis=0))**(1/2)
    error1 = np.abs(yp - ya)
    
    dic = {
        'ferror': ferror,
        'error': error1,
        'w': w,
        'bias': bias
    }
    return dic

In [4]:
data_all = np.loadtxt('../data/data_complete_tf_new.txt')

all_bin = np.vsplit(data_all, 6)
all_init = np.vstack([all_bin[i] for i in range(5)])
all_diff = np.vstack([all_bin[i+1]-all_bin[i] for i in range(5)])

In [5]:
# PATH = '../pickles/'

# result = skl_cv_quad(all_init, all_diff, kf=10)

# with open(PATH + 'skl_(99,99)_tvt.pkl', 'wb') as f:
#     pickle.dump(result, f)

# Neural Networks (tf.keras)
The computations were carried out using swarm. The materials are in ./HPC/3. Inference - NN/